# TODO
* Maximum a posteriori
* Regularised GMM

# *EM-algorithm*
## L'*EM-algorithm*: rationnel et principales idées
L'*EM algorithm* est une procédure d'estimation de distributions au maximum de vraisemblance pour des modèles à variables latentes (ou cachées). C'est une méthode itérative qui cherche à trouver des maximums locaux de la vraisemblance. Dans le modèle postulé dont on cherche à estimer les paramètres de la distribution. On y distingue deux familles de variables: 
* Les variables observées notées $x$
* Les variables inobservées (ou latentes, ou cachées) notées $z$ qui bien qu'inobservées contribuent au modèle (ex: les pondérations dans un Gaussian Mixture Model (GMM)).

En notant $x=\{x_{1}, \dots, x_{n}\}$ (idem pour $z$), une observation de $x$ est appelée *incomplete dataset* quand une observation de $(x,z)$ est appelée *complete dataset*.

La loi de densité de probabilité jointe paramétrée par un paramètre $\theta$ (et aussi appelée *complete data likelihood*) est notée $p(x,z|\theta)$. Dans le cas d'un GMM, $\theta$ engloberait tous les paramètes $\mu_i$, $\sigma_i$ et $\pi_i$. L'estimation du modèle au maximum de vraisemblance consiste à trouver le $\hat{\theta}$ maximisant l'*incomplete data likelihood* $p(x|\theta)$: 

$$\hat{\theta} = argmax_{\theta}p(x|\theta)$$

Trouver $\hat{\theta}$ constitue notre *learning problem*. Mais les variables latentes ne sont pas qu'un artifice, connaissant $\hat{\theta}$ et $x$ on veut pouvoir dire quelque chose de ces variables inobservées et en particulier de leur distribution. Calculer $p(z|x,\theta)$ constitue ce qu'on appelle l'*inference problem* (celui-ci étant par exemple particulièrement simple dans le cas des GMM).

Remarque: Il est plus souvent plus facile et donc plus fréquent de travailler avec la *log-likelihood* que la *likelihood* directement.

$logp(x|\theta)$ peut se voir comme $logp(x,z|\theta)$ marginalisée/sommée sur $z$. **La première est ainsi dénommée *marginalized log-likelihood* (ou parfois appelée *evidence*) et la seconde *complete data log-likelihood***.

L'*EM-algorithm* est pensé pour les cas où la maximisation de la *marginalized log-likelihood* $logp(x|\theta)$ est difficile mais celle de *complete data log-likelihood* $logp(x,z|\theta)$ est relativement plus simple. Problème, $z$ est par définition inobservée, on ne peut donc pas *plugger* de valeurs de $z$ dans $logp(x,z|\theta)$. Supposons en revanche qu'on possède une distribution $q$ pour $z$, cette distribution qu'on se donne ne pouvant être qu'une hypothèse (*guess*) ou connaissance à priori (*belief*) sur les $z$. Alors il devient potentiellement possible de calculer et de maximiser l'***expected*** *complete data log-likelihood*: 

$$\mathbb{E}_{z \sim q}[logp(x,z|\theta)] = \sum_{z}q(z)logp(x,z|\theta)$$

### L'*EM-algorithm* comme méthode variationnelle
L'*EM-algorithm* fait partie de la famille plus large des méthodes variationnelles. Ne pouvant directement maximiser notre objectif (ici la *marginalized log-likelihood* $logp(x|\theta)$), on va lui trouver une borne inférieure (ou plutôt une famille de bornes inférieures) avec laquelle il est plus simple de travailler et chercher à maximiser celle-ci. Ainsi, pour tout $\theta$ et toute distribution $q$ sur $z$:

$$
\begin{align}
logp(x|\theta) &= log[\sum_{z}p(x,z|\theta)] \; (loi \; des \; probabilités \; totales)\\
&= log[\sum_{z}q(z)\frac{p(x,z|\theta)}{q(z)}]\\
&\ge \sum_{z}q(z)log\frac{p(x,z|\theta)}{q(z)} = \mathcal{L}(q, \theta) \; (inégalité \; de \; Jensen)\\
\end{align}$$

Remarques: 
* Loi des probabilités totales : $p(x) = \sum_{z}p(x \cap z) = \sum_{z}p(x,z)$ par définition de la loi de probabilité jointe. 
* Sur les méthodes d'inférence variationnelles, voir *Pattern Recognition and Machine Learning* (Bishop, 2006), Chapitre 10.

### L'*EM-algorithm* comme un problème de maximisation-maximisation
On obtient donc une famille de bornes inférieures (de fonctions) $\mathcal{L}(q, \theta)$: pour chaque choix de distribution $q$, $logp(x|\theta)$ est inférieurement bornée par une fonction de $\theta$. L'idée de l'*EM-algorithm* va être de successivement maximiser cette borne inférieure par rapport à $q$ puis à $\theta$ (*coordinate ascent*). L'algorithme va ainsi consister à alterner deux étapes de maximisation:
* Pour un $\theta_{i}$ donné, on va chercher la distribution $q_{i+1}$ donnant la meilleure borne inférieure (fonction de $\theta$) à $logp(x|\theta_{i})$. Cette fonction va le plus souvent vouloir venir tangenter $logp(x|\theta_{i} - cf. ci-dessous)$ au point $\theta_{i}$.
* Pour une distribution $q_{i+1}$ donnée, on va chercher le paramètre $\theta_{i+1}$ qui maximise la fonction $\theta \mapsto \mathcal{L}(q_{i+1}, \theta)$
* On répète jusqu'à convergence.

La première et la seconde maximisation sont respectivement nommées *Expectation step* (ou *E-step*) et *Maximization step* (ou *M-step*). Cf. plus loin pour savoir pourquoi.

Cette démarche est permise par une propriété essentielle de l'*EM-algorithm* (car elle en assure la convergence) qui veut que la *marginalized log-likelihood* $logp(x|\theta)$ est monotone croissante sur la séquence de valeurs de paramètres $\theta_i$ obtenue: $\forall i, logp(x|\theta_{i+1}) \ge logp(x|\theta_{i})$

Remarque: L'*EM-algorithm* trouve donc sa justification dans le fait de ramener un problème de maximisation (de la *likelihood*) difficile à deux problèmes de maximisation plus simples.

### Propriétés de la borne inférieure  $\mathcal{L}(q, \theta)$
Intéressons nous d'abord aux propriétés de la borne inférieure:

$$\begin{align}
\mathcal{L}(q, \theta) &= \sum_{z}q(z)log\frac{p(x,z|\theta)}{q(z)}\\
&= \sum_{z}q(z)log\frac{p(z|x,\theta)}{q(z)} + logp(x|\theta) \; (définition \; probabilité \; conditionnelle)\\
&= -KL(q||p(z|x,\theta)) + logp(x|\theta)\\
\end{align}$$

Ecrit autrement, on voit que l'écart entre la *marginalized log-likelihood* $logp(x|\theta)$ et sa borne inférieure est donnée par la divergence de Kullback-Leibler entre la distribution $q$ postulée pour $z$ et la probabilité conditionnelle $p(z|x,\theta)$:

$$logp(x|\theta) = \mathcal{L}(q, \theta) + KL(q||p(z|x,\theta))$$

On déduit immédiatemment de cette égalité et des propriétés de la divergence de Kullback-Leibler que la meilleure borne inférieure $\mathcal{L}(q^{*}, \theta)$ à $\theta$ fixé est obtenue pour le choix de $q^{*}=p(z|x,\theta)$. On se trouve alors dans le cas d'égalité de l'inégalité obtenue plus haut à l'aide de l'inégalité de Jensen.

### Convergence de l'*EM-algorithm*
Reprenons l'étude de la convergence de l'algorithme, d'après l'inégalité ci-dessus, on a:

$$\forall q,\theta \; logp(x|\theta) \ge \mathcal{L}(q, \theta)$$

En particulier, à la suite de la seconde étape de la $(i+1)^{e}$ itération (*M-step*) on obtient $\theta_{i+1}$ tel que: 

$$logp(x|\theta_{i+1}) \ge \mathcal{L}(q_{i+1}, \theta_{i+1})$$

Or $\theta_{i+1}$ a été obtenu à la suite de la maximisation de la fonction $\theta \mapsto \mathcal{L}(q_{i+1}, \theta)$, on a donc en particulier: $\mathcal{L}(q_{i+1}, \theta_{i+1}) \ge \mathcal{L}(q_{i+1}, \theta_{i})$ On en déduit ainsi:

$$logp(x|\theta_{i+1}) \ge \mathcal{L}(q_{i+1}, \theta_{i}) \ge logp(x|\theta_{i})$$

La seconde inégalité provient de la positivité de la divergence de Kullback-Leibler et est en fait une égalité ($\mathcal{L}(q_{i+1}, \theta_{i}) = logp(x|\theta_{i})$) si et seulement si $q_{i+1}$ choisie telle que $q_{i+1}(z) = p(z|x,\theta_{i})$ (cf. ci-dessus). On en déduit ainsi sous les hypothèses faites jusqu'ici: 

$$\forall i, \; logp(x|\theta_{i+1}) \ge logp(x|\theta_{i})$$

Remarques: 
* Dans le prolongement de ces résultats, on montre facilement que s'il existe $(q^{*}, \theta^{*})$ réalisant un maximum global de la borne inférieure $\mathcal{L}(q, \theta)$, alors $\theta^{*}$ est aussi un maximum global de la *marginalized log-likelihood* $logp(x|\theta)$.
* Attention: Les maximas éventuellement découverts par l'*EM-algorithm* sont dans le cas général seulement des maximum locaux.

### Premiere maximisation - *E-step*
Comme vu plus haut, l'*EM-algorithm* peut se voir comme un double problème de maximisation, premier recherchant $q^{*}$ tel que:

$$q^{*} = argmax_{q}\mathcal{L}(q, \theta)$$

**Attention:** $q^{*}$ dépend de $\theta$ dans le cas général, on note alors $q^{*}=q^{*}_{\theta}$.

Or, on a montré ci-dessus en étudiant les propriétés de la borne inférieure que ce maximum est atteint pour $q^{*}_{\theta}(z) = p(z|x,\theta)$. La première étape de l'algorithme consiste ainsi à "simplement" calculer $J_{q^{*}_{\theta}}(\theta) = \mathcal{L}(q^{*}_{\theta}, \theta)$ que l'on va maximiser par rapport à $\theta$ à l'étape suivante. $\mathcal{L}(q^{*}_{\theta}, \theta)$ correspondant à une espérance (cf. ci-dessous), cette première étape consiste donc à calculer une espérance d'où son nom d'*Expectation step* (ou *E-step*) qui correspond au "E" de l'*EM-algorithm*.

L'espérance calculée lors de l'*E-step* où $q^{*}_{\theta}(z) = p(z|x,\theta)$ est:

$$J_{q^{*}_{\theta}}(\theta) = \mathcal{L}(q^{*}_{\theta}, \theta) = \sum_{z}q^{*}_{\theta}(z)log\frac{p(x,z|\theta)}{q^{*}_{\theta}(z)} = \mathbb{E}_{z \sim q{*}_{\theta}}[log\frac{p(x,z|\theta)}{q^{*}_{\theta}(z)}]$$

### Seconde maximisation - *M-step*
La seconde étape de l'EM-algorithme correspond à la recherche de $\theta^{*}$ par maximisation de la fonction $J_{q^{*}_{\theta}}:\theta \mapsto \mathcal{L}(q^{*}_{\theta}, \theta)$ obtenue à l'étape précédente. Cette étape correspond au "M" de l'*EM-algorithm* et est appelé *Maximization step* ou *M-step*.

### Résumé
En pratique l'*EM-algorithm* se résume après initialisation d'un $\theta$ à $\theta_0$, à la répétition jusqu'à convergence (décrétée à partir du moment où on arrive plus à améliorer significativement la *marginalized log-likelihood* $logp(x|\theta)$) des deux étapes suivantes (on décrit ici la $(i+1)^{e}$ itération): 
* ***Expectation step*** : Calcul de $J_{q_{i+1}}(\theta)=\mathbb{E}_{z \sim q_{i+1}}[log\frac{p(x,z|\theta)}{q_{i+1}(z)}]$ avec $q_{i+1}(z) = p(z|x,\theta_{i})$.
* ***Maximization step*** : Calcul de $\theta_{i+1} = argmax_{\theta}J_{q_{i+1}}(\theta)$

L'usage de l'*EM-algorithm* est particulièrement justifiée quand *E-step* et *M-step* sont relativement faciles (notamment par rapport à une maximisation directe de la *likelihood*). Dans le cas des GMM par exemple, l'*EM-algorithm* est particulièrement adapté car chacune des deux étapes possède une *closed-form solution*. Toutefois: 
* La maximisation à réaliser au *M-step* peut ne pas être triviale et exiger de recourir à des techniques d'optimisation dédiées (*gradient descent*, etc.). On peut également seulement se contenter d'un $\theta_{i+1}$ qui améliore $J_{q_{i+1}}(\theta)$ sans en être le maximum (*generalized EM*) restant sûrs que la *likelihood* va croître de façon monotone.
* Le calcul d'espérance du *E-step* peut également ne pas être facile à faire. On peut alors décider de restreindre le choix de la distribution $q$ à une famille de distributions $\mathcal{Q}$ pour laquelle le calcul de $J(\theta)$ est aisé. Notre borne inférieure ne sera alors plus forcément la meilleure possible (cas d'égalité) mais plus lâche (*looser*) et $q^{*}$ sera choisie telle que $q^{*}=argmin_{q \in \mathcal{Q}}KL(q|p(z|x,\theta))$ en espérant que ce nouveau problème soit facilement soluble.

## Annexe
### *Coordinate ascent*
Le terme de *coordinate ascent* (resp. *coordinate descent*) sert à désigner un algorithme maximisant (resp. minimisant) une fonction en ne la maximisant à chaque itération que suivant une seule coordonnée (direction) ou un sous-ensemble de coordonnées. Le terme s'applique dans le cas de l'*EM-algorithm* du fait qu'on maximise une fonction $(q, \theta) \mapsto \mathcal{L}(q, \theta)$ successivement suivant $q$ (*E-step*) puis $\theta$ (*M-step*).

### Inégalité de Jensen
Soient $f$ une fonction $f:\mathbb{R} \rightarrow \mathbb{R}$ convexe et $x$ une variable aléatoire, on a alors: 

$$\mathbb{E}[f(x)] \ge f(\mathbb{E}[x])$$

Si $f$ est de plus strictement convexe, alors le cas d'égalité implique que $x$ est contante.

Astuce mnémotechnique pour se rappeler du sens de l'inégalité: $Var(x) = \mathbb{E}[x^2]- (\mathbb{E}[x])^2 \geq 0$ et $f: x \mapsto x^2$ convexe.

### Divergence de Kullback-Leibler
On définit la divergence de Kullback-Leibler de deux distributions $p$ et $q$ comme: 

$$KL(p||q)=\sum_{x}p(x)log\frac{p(x)}{q(x)}$$

Pour faire simple et rapide, la divergence de Kullback-Leibler de deux distributions $p$ et $q$ est une "mesure" de leur dissimilarité. On peut citer les deux de ses propriétés utiles ici: elle est toujours positive ou nulle (inégalité de Gibbs) et n'est nulle que si les deux distributions sont strictement identiques. Attention il ne s'agit ni d'une distance ni d'une métrique, la *KL-divergence* n'étant pas symétrique (sans parler de l'absence d'inégalité triangulaire).

# *Gaussian mixture models* (GMM)
Les GMM sont une famille de modèles faisant l'hypothèse que la distribution à modéliser peut s'écrire comme une combinaison linéaire de $k$ gaussiennes:

$$p(x)=\sum_{i=1}^{k}\pi_{i}\mathcal{N}(x |\mu_{i}, \Sigma_{i})$$

Les paramètres $\pi_{i}$ reflètent la contribution de chaque gaussienne à l'ensemble et possèdent les deux propriétés suivantes:
* $\sum_{i}\pi_{i}=1$ Il suffit pour le montrer de sommer l'égalité précédente sur $x$.
* $0 \le \pi_{i} \le 1$
On remarque que les paramètres $\pi_{i}$ satisfont ainsi les condition minimales requises pour constituer une distribution de probabilités.

L'estimation d'un GMM consiste à estimer les paramètres $\pi_{i}$, $\mu_{i}$ et $\Sigma_{i}$ qu'on désigne collectivement par le paramètre $\theta$. Pour une *likelihood* $p(x|\theta)=\prod_{i=1}^{n}p(x_{i}|\theta)$, la *log-likelihood* plus simple à manipuler, d'un GMM va ainsi s'écrire:

$$logp(x|\theta)=\sum_{i=1}^{n}log(\sum_{j=1}^{k}\pi_{i}f_{\mu_{i}, \Sigma_{i}}(x_{j}))$$

On voit à la présence d'une somme dans le log que la maximisation directe de la *likelihood* risque de ne pas être simple. L'estimation des GMM au maximum de vraisemblance recourt ainsi à l'*EM-algorithm*. On introduit pour celà des variables latentes inobservées $z$ de telle façon que le GMM $p(x)$ corresponde à la distribution marginale de la distribution jointe $p(x,z)=p(z)p(x|z)$.

## Variables latentes d'un GMM
Remarque: Par souci de clarté, $x$ désigne l'ensemble des observations dans cette partie.

L'utilisation de l'*EM-algorithm* a requis l'introduction d'une variable latente $z$. Cette variable prend ses valeurs dans l'ensemble des vecteurs à $k$ composantes dont une seule est égale à $1$ tandis que l'ensemble des autres sont égales à $0$. La variable $z$ ne peut ainsi prendre que $k$ valeurs, la probabilité qu'elle puisse prendre la valeur du vecteur dont la $i^e$ composante est à $1$ étant $\pi_i$. Dit autrement, la loi suivie par $z$ est une loi catégorielle (loi de Bernoulli généralisée à plus de deux catégories), loi de probabilité discrète dont la fonction de masse s'écrit $p(z=z_{i})=\pi_i$ avec $z_{i}$ le vecteur de $\mathbb{R}^k$ dont la $i^e$ composante est égale à $1$ tandis que les autres sont à $0$.

L'introduction de cette variable latente donne une autre interprétation au GMM: tout se passe comme si chacune des observations n'a été généré que par une seule des gaussiennes composant le modèle. A chaque nouvelle observation, tout se passe comme si:
* On effectuait dans un premier temps un tirage de la variable $z$ qui nous donne le vecteur $z_i$ avec une problabilité $\pi_i$
* On effectuait dans un second temps un tirage de la variable $x$ dont on suppose à cette étape qu'elle se distribue comme la $i^e$ gaussienne de la mixture.

Il découle immédiatement de la seconde étape du raisonnement:

$$p(x|z=z_{i}) = \mathcal{N}(x |\mu_{i}, \Sigma_{i})$$

On retrouve ainsi la spécification du modèle: 

$$p(x)=\sum_{i=1}^{k}p(z=z_{i})p(x|z=z_{i})=\sum_{i=1}^{k}\pi_{i}\mathcal{N}(x |\mu_{i}, \Sigma_{i})$$

On insiste ici sur le "tout se passe comme si". C'est dans cette vue de l'esprit que la variable $z$ prend tout son sens: la modélisation du problème fait appel à cette variable mais qui est cependant inobservable en pratique.

Remarques: 
* Plus $\pi_i$ sera élevée, plus la $i^e$ gaussienne de la mixture sera choisie souvent et graphiquement, plus le cluster associé à cette composante sera dense.
* On introduit la dernière quantité non encore abordée $p(z|x)$ appelée aussi *responsibility* qu'on déduit aisément des autres quantités déjà introduites à l'aide de la formule de Bayes: 

$$p(z=z_{i}|x) = \frac{\pi_{i}\mathcal{N}(x |\mu_{i}, \Sigma_{i})}{\sum_{j=1}^{k}\pi_{j}\mathcal{N}(x |\mu_{j}, \Sigma_{j})} = \gamma_{i}$$

$p(z=z_i)=\pi_i$ peut ainsi être vue comme la *prior probability* de $z=z_i$ et la *responsibility* comme la *posterior probability* (probabilité réévaluée après observation des données) associée. De manière générale à chaque étape de l'*EM-algorithm* qui consiste en deux maximisations successives, $p(z|x)$ est à la fois la distribution issue de la première et celle utilisée dans la seconde. On comprend dès lors l'importance du rôle que vont jouer les *responsibilities* dans l'*EM-algorithm* appliqué aux GMM.

## Estimation d'un GMM avec l'*EM-algorithm*
En s'en tenant à une description générale de l'*EM-algorithm*, développons les deux étapes de la $i^e$ itération qui pour plus de clarté sera notée en exposant:

### *E-step* 
Calcul de $J_{q^{(i)}}(\theta)=\mathbb{E}_{z \sim q^{(i)}}[log\frac{p(x,z|\theta)}{q^{(i)}(z)}]$ avec $q^{(i)}(z) = p(z|x,\theta^{(i-1)})$

D'après le paragraphe précédent $p(z|x,\theta^{(i-1)})$ est connue, sa fonction de masse correspond aux *responsibilities*. Ainsi:

$$p(z=z_{j}|x,\theta^{(i-1)}) = \gamma_{j}(x) = \frac{\pi_{j}^{(i-1)}\mathcal{N}(x |\mu_{j}^{(i-1)}, \Sigma_{j}^{(i-1)})}{\sum_{u=1}^{k}\pi_{u}^{(i-1)}\mathcal{N}(x |\mu_{u}^{(i-1)}, \Sigma_{u}^{(i-1)})}$$

Avec $\mathcal{N}(x |\mu_{j}^{(i-1)}, \Sigma_{j}^{(i-1)}) = \prod_{u=1}^{n}\mathcal{N}(x_{u} |\mu_{j}^{(i-1)}, \Sigma_{j}^{(i-1)})$.

$J_{q^{(i)}}(\theta)$ prend donc la forme relativement simple: 

$$\begin{align}
J_{q^{(i)}}(\theta) &= \sum_{j=1}^{k}\gamma_{j}log(\frac{p(x,z_{j}|\theta^{(i-1)})}{\gamma_{j}})\\ 
&= \sum_{j=1}^{k}\gamma_{j}log(\frac{p(z_{j}|\theta^{(i-1)})p(x|z_{j},\theta^{(i-1)})}{\gamma_{j}})\\
&= \sum_{j=1}^{k}\gamma_{j}log(\frac{\pi_{j}\mathcal{N}(x |\mu^{(i-1)}, \Sigma^{(i-1)})}{\gamma_{j}})\\
\end{align}$$

### *M-step*
On maximise dans cette étape la fonction $\theta \mapsto J_{q^{(i)}}(\theta)$ obtenue à l'étape précédente en écrivant simplement les conditions du premier ordre. La résolution des équations obtenues donnent ainsi les formules d'*update* de $\pi^{(i)}$, $\mu^{(i)}$ et $\Sigma^{(i)}$ qui seront utilisées dans la $(i+1)^e$ étape de l'algorithme. On obtient ainsi après calcul: 
* $\mu_{j}^{(i)} = \frac{1}{n_{j}}\sum_{u=1}^{n}\gamma_j(x_{u})x_{u}$ avec $n_{j}= \sum_{u=1}^{n}\gamma_{j}(x_{u})$ 
* $\Sigma_{j}^{(i)} = \frac{1}{n_{j}}\sum_{u=1}^{n}\gamma_j(x_{u})(x_{u}-\mu_{j}^{(i)})(x_{u}-\mu_{j}^{(i)})^{\top}$ avec $n_{j}= \sum_{u=1}^{n}\gamma_{j}(x_{u})$
* $\pi_{j}^{(i)} = \frac{1}{n}\sum_{u=1}^{n}{\gamma_{j}(x_{u})}$

On peut reformuler le déroulé de ces deux étapes pour retrouver l'algorithme usuellement présenté pour les GMM:
* Pour chacune des $n$ observations, on calcule son jeu de $k$ *responsibilities* qui correspond aux probabilités qu'elles aient pu être générée par chacune des $k$ gaussiennes obtenues à l'étape précédente. 
* On rééstime les moyennes et matrices de covariance de chacune des $k$ gaussiennes, il s'agit des estimateurs classiques mais où chaque observation est pondérée par la *responsibility* de la gaussienne considérée: plus il est considéré probable que la gaussienne considérée ait généré l'observation, plus on donne de poids à celle-ci dans l'estimation des paramètres.
* Pour chaque gaussienne $j$, on update son poids $\pi_j$ avec la valeur moyenne de ses *responsibilities* $\gamma_{j}$.

## Les GMM comme algorithme de clustering
On a considéré jusqu'ici les GMM comme un simple modèle de densité de probabilité un peu difficile à estimer. On peut toutefois exploiter le fait que le modèle consiste en la réunion de $k$ densités de probabilités pour obtenir un partitionnement des observations, les paramètres $\pi_i$ donnant une idée des populations relatives des clusters:
* Chaque observation est assignée au cluster/à la gaussienne qui présente la plus haute valeur de *responsibility*. L'étude des *responsibilities* nous donne une idée de la "certitude" avec laquelle l'assignation a été faite. C'est pour cela qu'on parle pour les GMM de *soft assignment* par opposition au *hard assignement* qu'on rencontre dans d'autres algorithmes de partitionnement comme le $k$*-means*.
* L'assignation d'une nouvelle observation peut se faire par le calcul des distance de Mahalanobis de l'observation à chacune des gaussiennes ou un calcul de *responsibilities* (équivalent ?).

### Lien avec l'algorithme du $k$*-means*
On peut montrer que l'algorithme du $k$*-means* correspond exactement à un cas limite des GMM où les matrices de covariance des différentes gaussiennes sont notamment contraintes à être des multiples de l'identités (gaussiennes de forme sphérique). Cette restriction nous permet d'ailleurs de comprendre les limitations $k$*-means* quant à la topologie des clusters que l'algorithme est capable de correctement extraire. Ces éléments nous permettent de comprendre pourquoi les GMM sont parfois présentés comme une généralisation plus flexible du $k$*-means*.

## Initialisation
De la même manière que le $k$*-means* qui du fait de sa *greediness* peut tomber dans des maximas locaux et donc de ce fait sensible à l'initialisation, les GMM sont aussi sensibles à l'initialisation de l'*EM-algorithm* (qui peut lui aussi tomber dans des maximas locaux de la vraisemblance).

On distingue globalement deux méthodes d'initialisation pour la calibration des GMM:
* Une méthode aléatoire: on sélectionne $k$ points au hasard qui vont permettre d'initialiser les moyennes $\mu$, les matrices de covariance $\Sigma$ sont initialisées avec la matrice de covariance des données et les poids $\pi$ initialisés à $\frac{1}{n}$ (ou au hasard ?).
* Utilisation d'un $k$*-means*: on initialise des moyennes $\mu$ avec les poids proches de centroïdes, les poids $\pi$ et les matrices de covariance $\Sigma$ sont initialisés sur les clusters obtenus par le $k$*-means*.

## Risques de divergence
L'utilisation du maximum de vraisemblance pour la calibration de GMM avec $k>1$ s'expose à la présence de singularités. Supposons pour simplifier la matrice de covariance diagonale et de paramètre $\sigma$ et qu'une gaussienne se retrouve centrée sur un point isolé, sur du bruit. Faire tendre $\sigma$ vers zéro nous permettra alors de toujours faire croire la vraisemblance. Ces singularités donnent également un exemple du sévère *overfitting* auxquelles les méthodes du maximum de vraisemblance peuvent conduire. Ces singularités n'existent pas dans les approches bayésiennes du même problème. En attendant, il est nécessaire de se doter de garde-fous dans l'implémentation de l'*EM-algorithm*. 